**Depósito: Somente valores positivos. Deve ser retornado em extrato

**Saques: Somente 3 por dia. Máximo de 500 por saque. Caso não tenha saldo o sistema exibe uma mensagem. Deve ser retornado os saques em extrato

**Extrato: Depósitos e Saques e no fim da listagem o saldo da conta. Deve-se usar o R$.

**Estabelecer um limite de 10 transações diárias para a conta 

**Se o usuário tentar fazer uma transação após atingir o limite, deve ser informado que ele excedeu o número de transações permitidas para aquele dia.

**Mostre no extrato, a data e hora de todas as transações



'''
Depósito: Somente valores positivos. Deve ser retornado em extrato
Saques: Somente 3 por dia. Máximo de 500 por saque. Caso não tenha saldo o sistema exibe uma mensagem. Deve ser retornado os saques em extrato
Extrato: Depósitos e Saques e no fim da listagem o saldo da conta. Deve-se usar o R$.

'''
         


In [58]:
from datetime import datetime
from random import randrange
import pytz


TZ_FORTALEZA = pytz.timezone('America/Fortaleza')
LIMITE_DIARI0 = 500
QTD_SAQUE_DIARIO = 3
saldo = 0
msg = []
dt_operacao = datetime.now(tz=TZ_FORTALEZA)
count = 0
users = []
banco_contas = []



def criar_usuario(user):
    global users
    if user['cpf'] in [user['cpf'] for user in users]:
       return f'CPF {user["cpf"]} ja existe.'
    else:
       users.append(user)
       return f'Usuario {user["cpf"]} criado com sucesso.'
    
def criar_conta_corrente(cpf):  
    global banco_contas  
    prefixo = randrange(1000, 9999)
    dv = randrange(0, 9) 
    cpf_exist = [user for user in users if user['cpf'] == cpf]     
    if cpf_exist is not None:
       # Verificar se o usuário já possui uma conta
       exist = cpf_exist[0].get('conta')
       if exist is not None:
          return f'Usuário já possui conta. {cpf_exist[0]["conta"]}'
       else:
          print('Usuário não possui conta. Criando...')
          # vou usar o filter para verificar se  no banco_contas existe a conta
          conta = list(filter(lambda user: user['conta'] == f'{prefixo}-{dv}', banco_contas))          
          while not conta:
                prefixo = randrange(1000, 9999)
                dv = randrange(0, 9) 
                cpf_exist[0].update({'conta': f'{prefixo}-{dv}', 'saldo': 0, 'historico': []})
                conta = list(filter(lambda user: user['conta'] == f'{prefixo}-{dv}', banco_contas))
                return f'Conta criada com sucesso. {prefixo}-{dv}' 
          
    else:
        return 'CPF inválido.'      

def deposito(valor, conta):
    # Condição para saber se a QTD_OPERACOES_DIARIAS foi atingida       
    operacoes = {}
    contador = 0

    # Vou buscar a quantidade atual de operações por dia
    for user in users:
        
        if user['conta'] == conta: 
           
          
           if user['historico'] is not None:              
              
              for value in user['historico']:                            
                  if value.get('dt_operacao') == dt_operacao.strftime("%d/%m/%Y"):
                     contador += 1

              if contador > 10:
                 return 'Limite de operações por dia atingido. Tente amanhã. '

              else:  
                 # Adicionando o valor no dicionário                
                 operacoes = {'dt_operacao': dt_operacao.strftime("%d/%m/%Y"), 'valor': valor, 'tipo': 'deposito'}
                 user['historico'].append(operacoes)
                 user.update({'saldo': user['saldo'] + valor})                
                 return operacoes  
              
           else:
               operacoes = {'dt_operacao': dt_operacao.strftime("%d/%m/%Y"), 'valor': valor, 'tipo': 'deposito'}
               user.update({'saldo': valor, 'historico': [operacoes]})                
               return operacoes                        
                         
        else:
            return 'Conta inválida.'       
    
    
    
def saque(conta, valor):
    # Vou realizar as mesmas operações que foram feitas no Depósito mas verificando se há saldo suficiente
    operacoes = {}
    contador = 0
    saldo = 0

    for user in users: # Vou percorrer a lista de dicionário users

        if user['conta'] == conta: # Verificando se a conta existe
           
           saldo = user['saldo'] # Pegando o saldo atual

           for value in user['historico']: # contando o historico pelo dia atual
               if value.get('dt_operacao') == dt_operacao.strftime("%d/%m/%Y"):
                  contador += 1

           if contador >= 10: # Verificando se a quantidade de operações diarias foi atingida
               return 'Limite de operações por dia atingido. Tente amanhã. '

           elif saldo < valor: # Verificando se o valor do saque é maior que o saldo
               return 'Saldo insuficiente.'
                   
           else: # Vou realizar o saque caso todas as condições sejam atendidas
               operacoes = {'dt_operacao': dt_operacao.strftime("%d/%m/%Y"), 'valor': valor, 'tipo': 'deposito'}
               user.update({'saldo': user['saldo'] - valor})
               user['historico'].append(operacoes)
               return operacoes    
           
        else: # Vou retornar uma mensagem caso a conta seja inválida
            return 'Conta inválida.'      

def extrato(conta):
    # Fazer um loop para saber o histórico da conta no dia atual    
    for user in users:
        if user['conta'] == conta:
           return user
               


while True:
        print(
            """
            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] Criar Conta Corrente
            [3] Depositar
            [4] Sacar
            [5] Extrato
            [6] Sair

            """ 
        ) 

        option = int(input("Opção: "))

        if option == 6:
            print('Saindo...')
            break
        elif option == 1:
            name_user = input('Nome do Usuário: ')
            data_nasc = input('Data de Nascimento: Ex: 01/01/2000: ')
            cpf = input('CPF: Obs: somente números: ')
            logradouro = input('Logradouro: Ex: Rua das Acaúbas, 123, Vila Nova, São Paulo, SP, 12345-678 \n')
            dt_criacao = dt_operacao
            user = {'cpf': cpf,'nome': name_user, 'data_nasc':data_nasc, 'logradouro':logradouro}
            print(criar_usuario(user))
        elif option == 2:
             cpf = input('CPF: Obs: somente números: ')
             print(criar_conta_corrente(cpf))             
        elif option == 3:
            valor = float(input('Digite o valor para depósito: ')) 
            conta = input('Digite a conta: Ex: 1398-7: ')
            print(deposito(valor, conta))
        elif option == 4:
            valor = float(input('Digite o valor para saque: ')) 
            conta = input('Digite a conta: Ex: 1398-7: ')
            print(saque(conta, valor))      
        elif option == 5:
            conta = input('Digite a conta: Ex: 1398-7: ')
            tamanho = 5
            result = extrato(conta)            
            # Imprime as 5 ultimas operações
            x = slice(5)
            for value in result['historico'][x]: # Imprimindo o extrato com os 5 ultimos movimentos
                print(f'{value["dt_operacao"]} - {value["valor"]} - {value["tipo"]}')
           
           
            print(f'O saldo total: R$ {result["saldo"]}')
             
        else:
            print('Opção inválida. Tente novamente.')                



            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] Criar Conta Corrente
            [3] Depositar
            [4] Sacar
            [5] Extrato
            [6] Sair

            
Usuario 41930770391 criado com sucesso.

            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] Criar Conta Corrente
            [3] Depositar
            [4] Sacar
            [5] Extrato
            [6] Sair

            
Usuário não possui conta. Criando...
Conta criada com sucesso. 3052-4

            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] Criar Conta Corrente
            [3] Depositar
            [4] Sacar
            [5] Extrato
            [6] Sair

            
{'dt_operacao': '26/09/2024', 'valor': 100.0, 'tipo': 'deposito'}

            ######MENU######

            Selecione uma Opção:
            [1] Criar Usuário
            [2] 